In [8]:
!pip install -r /content/requirements.txt

In [9]:
from dotenv import load_dotenv
import os
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS,DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [22]:
load_dotenv()
openai_api_key = "sk-proj-N7E1cs97YAu6EmJTFsPqT3BlbkFJg2DwKsXrgsuW9XY3y82q"
Model = 'gpt-3.5-turbo'


In [23]:
model = ChatOpenAI(api_key=openai_api_key,model=Model)
model.invoke('llama3?')

AIMessage(content='I\'m not sure what you mean by "llama3." Could you please provide more context or clarify your question?', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 11, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-04ed6d92-f5f6-43d4-bc7f-52f60203032e-0')

In [15]:
parser = StrOutputParser()
chain = model|parser

In [18]:
chain.invoke('what is llama3?')

'There is no specific definition for "llama3." It is possible that it could be a typo or a made-up term. If you have any additional context or information, please provide it so I can better assist you.'

In [19]:
file_loader = PyPDFLoader('llama3.pdf')
page = file_loader.load_and_split()
len(page)

37

In [20]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
pages = spliter.split_documents(page)
pages[3]

Document(page_content='state-of-the-art models using publicly available\ndatasets exclusively, without resorting to\nproprietary and inaccessible datasets. In\nparticular, LLaMA-13B outperforms GPT-3', metadata={'source': 'llama3.pdf', 'page': 0})

In [26]:
vector_storage = FAISS.from_documents(pages,OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = vector_storage.as_retriever()

In [27]:
question_template = """
your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context:{context}
question:{question}

"""

In [28]:
prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'
              ))


your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context: Here is the context to use
question: Answer this question based on the context




In [29]:
result = RunnableParallel(context= retriever,question = RunnablePassthrough())

In [30]:
chain = result |prompt | model | parser

In [31]:
chain.invoke('what is llama3?')

'Based on the context provided, llama3 appears to refer to a model or system that is being compared to other models like GPT-3, Chinchilla, and PaLM in terms of performance on benchmarks. It is mentioned that llama3 models, such as LLaMA-13B and LLaMA-65B, outperform GPT-3 and are competitive with other models despite being smaller in size.'

In [32]:
retriever.invoke('what is llama3?')

[Document(page_content='portantly, the LLaMA-13B is also competitive on\nthese benchmarks with GPT-3 and Chinchilla, de-\nspite being 5-10 ×smaller. This model runs on a\nsingle V100 GPU during inference.', metadata={'source': 'llama3.pdf', 'page': 4}),
 Document(page_content='particular, LLaMA-13B outperforms GPT-3\n(175B) on most benchmarks, and LLaMA-65B\nis competitive with the best models, Chinchilla-\n70B and PaLM-540B. We release all our', metadata={'source': 'llama3.pdf', 'page': 0}),
 Document(page_content='notably, LLaMA-13B outperforms GPT-3 while\nbeing more than 10 ×smaller, and LLaMA-65B is\ncompetitive with Chinchilla-70B and PaLM-540B.\nUnlike previous studies, we show that it is possible', metadata={'source': 'llama3.pdf', 'page': 10}),
 Document(page_content='to 65B parameters with competitive performance\ncompared to the best existing LLMs. For instance,\nLLaMA-13B outperforms GPT-3 on most bench-\nmarks, despite being 10 ×smaller. We believe that', metadata={'source